In [1]:
! python3 --version

Python 3.8.12


In [2]:
import pandas as pd
import numpy as np
import os
import re
import functools

pd.__version__, np.__version__

('1.4.1', '1.21.2')

In [3]:
project_dir = '/Users/gurdeep/Documents/tb2/DSMP/G32-Butterfly-Data'
module_dir = project_dir+'/modules'

dataset_filepath = '/Users/gurdeep/Documents/tb2/DSMP/Licensed vehicles by postcode district and body type.xlsx'

os.chdir(project_dir)
current_working_dir = os.getcwd()
print("Current working dir:", current_working_dir)
print("All paths correct:", os.path.isdir(project_dir) == os.path.isdir(module_dir) == os.path.isfile(dataset_filepath) == True)

Current working dir: /Users/gurdeep/Documents/tb2/DSMP/G32-Butterfly-Data
All paths correct: True


In [4]:
# importing preprocessing module
os.chdir(module_dir)
import Preprocessing
os.chdir(current_working_dir)

## Loading File

In [5]:
extract_configs = [
        {'io': dataset_filepath, 'sheet_name':'2021', 'header':[0,1], 'skiprows':6, 'skipfooter':13},
        {'io': dataset_filepath, 'sheet_name':'2020', 'header':[0,1], 'skiprows':6, 'skipfooter':13},
        {'io': dataset_filepath, 'sheet_name':'2019', 'header':[0,1], 'skiprows':6, 'skipfooter':13},
        {'io': dataset_filepath, 'sheet_name':'2018', 'header':[0,1], 'skiprows':6, 'skipfooter':12},
        {'io': dataset_filepath, 'sheet_name':'2017', 'header':[0,1], 'skiprows':6, 'skipfooter':8},
        {'io': dataset_filepath, 'sheet_name':'2016', 'header':[0,1], 'skiprows':6, 'skipfooter':8},
        {'io': dataset_filepath, 'sheet_name':'2015', 'header':[0,1], 'skiprows':6, 'skipfooter':8},
        {'io': dataset_filepath, 'sheet_name':'2014', 'header':[0,1], 'skiprows':6, 'skipfooter':8},
    ]

In [6]:
dfs = {
    config['sheet_name']:pd.read_excel(**config) for config in extract_configs
}

for sheet_name, df in dfs.items():
    print(sheet_name, "\n")
    df.info()
    print("\n"+"-"*50+"\n")

2021 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2903 entries, 0 to 2902
Data columns (total 12 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   (Unnamed: 0_level_0, Postcode District)  2903 non-null   object 
 1   (2021 Q1 , Cars)                         2898 non-null   object 
 2   (2021 Q1 , Motorcycles)                  2898 non-null   object 
 3   (2021 Q1 , All others)                   2898 non-null   object 
 4   (2021 Q1 , All others.1)                 0 non-null      float64
 5   (2021 Q2, Cars)                          2898 non-null   object 
 6   (2021 Q2, Motorcycles)                   2898 non-null   object 
 7   (2021 Q2, All others)                    2898 non-null   object 
 8   (2021 Q2, All others.1)                  0 non-null      float64
 9   (2021 Q3, Cars)                          2899 non-null   object 
 10  (2021 Q3, Motorcycles)                   

## Renaming columns

In [7]:
def multiIndexRenameMapper(column_names: pd.MultiIndex):
    assert isinstance(column_names, pd.MultiIndex)
    mapper = {}
    for col in column_names:
        assert isinstance(col, tuple)
        assert len(col) == 2
        if any(['Postcode' in c for c in col]): 
            mapper[col] = "_".join(col[-1].strip().lower().split(" "))
        else:
            first, second = col[0].strip().lower().split(" "), col[1].strip().lower().split(" ")
            assert len(first) == 2
            assert len(second) in [1,2]
            first = first[-1] # just taking quarter and ignoring year
            
            if len(second) == 2 and "all" in second[0] and "others" in second[1]:
                second = second[1] # if r'all others*' is present then take only r'others*' as second
            else:
                second = "".join(second)
                
            mapper[col] = first + "_" + second

    return mapper

def renameDataFrame(df: pd.DataFrame, year):
    assert isinstance(df, pd.DataFrame)
    mapper = multiIndexRenameMapper(df.columns)
    new_df = pd.DataFrame()
    
    for prev_col, new_col in mapper.items():
        new_df[new_col] = df[prev_col]
    
    new_df['year'] = year    
    return new_df

In [8]:
dfs = {year: renameDataFrame(dfs[year], year) for year in dfs.keys()}

for sheet_name, df in dfs.items():
    print(sheet_name, "\n")
    df.info()
    print("\n"+"-"*50+"\n")

2021 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2903 entries, 0 to 2902
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   postcode_district  2903 non-null   object 
 1   q1_cars            2898 non-null   object 
 2   q1_motorcycles     2898 non-null   object 
 3   q1_others          2898 non-null   object 
 4   q1_others.1        0 non-null      float64
 5   q2_cars            2898 non-null   object 
 6   q2_motorcycles     2898 non-null   object 
 7   q2_others          2898 non-null   object 
 8   q2_others.1        0 non-null      float64
 9   q3_cars            2899 non-null   object 
 10  q3_motorcycles     2899 non-null   object 
 11  q3_others          2899 non-null   object 
 12  year               2903 non-null   object 
dtypes: float64(2), object(11)
memory usage: 295.0+ KB

--------------------------------------------------

2020 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2

## Merging All dataframes

In [9]:
print("Expected dataframe length after concat:", sum([df.shape[0] for df in dfs.values()]))

Expected dataframe length after concat: 23240


In [10]:
df = pd.concat(list(dfs.values())) 
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23240 entries, 0 to 2905
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   postcode_district  23240 non-null  object 
 1   q1_cars            23133 non-null  object 
 2   q1_motorcycles     23133 non-null  object 
 3   q1_others          23133 non-null  object 
 4   q1_others.1        0 non-null      float64
 5   q2_cars            23137 non-null  object 
 6   q2_motorcycles     23137 non-null  object 
 7   q2_others          23137 non-null  object 
 8   q2_others.1        0 non-null      float64
 9   q3_cars            23218 non-null  object 
 10  q3_motorcycles     23218 non-null  object 
 11  q3_others          23218 non-null  object 
 12  year               23240 non-null  object 
 13  q3_others.1        0 non-null      float64
 14  q4_cars            20309 non-null  object 
 15  q4_motorcycles     20309 non-null  object 
 16  q4_others          2030

In [11]:
df.head()

,postcode_district,q1_cars,q1_motorcycles,q1_others,q1_others.1,q2_cars,q2_motorcycles,q2_others,q2_others.1,q3_cars,q3_motorcycles,q3_others,year,q3_others.1,q4_cars,q4_motorcycles,q4_others
0,AB1,49,38,50,NaN,50,38,50,NaN,49,38,48,2021,NaN,NaN,NaN,NaN
1,AB10,8150,269,688,NaN,8174,313,693,NaN,8150,304,680,2021,NaN,NaN,NaN,NaN
2,AB11,6476,204,1288,NaN,6485,230,1324,NaN,6437,239,1314,2021,NaN,NaN,NaN,NaN
3,AB12,15056,507,2781,NaN,15088,583,2812,NaN,14961,567,2880,2021,NaN,NaN,NaN,NaN
4,AB13,1885,48,132,NaN,1897,48,140,NaN,1905,51,148,2021,NaN,NaN,NaN,NaN


## Rebuilding DataFrame

In [12]:
def DataframeSplits(on:[str], columns):
    splits = []
    for reg_str in on:
        splits.append(list(filter(lambda s: re.match(reg_str, s), columns)))
    return splits

def processDataFrames(df, mapper, quarter):
    df = pd.DataFrame(df)
    df.rename(columns=mapper, inplace=True)
    df['quarter'] = quarter
    return df

def rebuildDataFrame(df, on):
    # findind columns names of each slice of dataframe
    splits = DataframeSplits(on, df.columns.tolist())
    rename_mappers = [(cols[0][1], {c: c.split("_")[-1] for c in cols}) for cols in splits]
    remaining_cols = list(set(df.columns.tolist()) - set(functools.reduce(lambda a,b: a+b, splits)))
    splits = [split+remaining_cols for split in splits]
    
    # renaming each dataframe
    dfs = [processDataFrames(df[cols], mapper[1], mapper[0]) for cols, mapper in zip(splits, rename_mappers)]
    
    # concatenating all dfs
    return pd.concat(dfs).reset_index(drop=True)
    

# chekcing rebuilt dataframe with count of each year
rebuildDataFrame(df, ['^q1', '^q2', '^q3', '^q4']).groupby(['year', 'quarter'])['postcode_district'].agg(['count'])

count
year quarter       
2014 1         2906
     2         2906
     3         2906
     4         2906
2015 1         2905
     2         2905
     3         2905
     4         2905
2016 1         2910
     2         2910
     3         2910
     4         2910
2017 1         2910
     2         2910
     3         2910
     4         2910
2018 1         2903
     2         2903
     3         2903
     4         2903
2019 1         2902
     2         2902
     3         2902
     4         2902
2020 1         2901
     2         2901
     3         2901
     4         2901
2021 1         2903
     2         2903
     3         2903
     4         2903

In [13]:
df = rebuildDataFrame(df, ['^q1', '^q2', '^q3', '^q4'])
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92960 entries, 0 to 92959
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   cars               89797 non-null  object 
 1   motorcycles        89797 non-null  object 
 2   others             89797 non-null  object 
 3   others.1           0 non-null      float64
 4   postcode_district  92960 non-null  object 
 5   year               92960 non-null  object 
 6   quarter            92960 non-null  object 
dtypes: float64(1), object(6)
memory usage: 5.0+ MB


,cars,motorcycles,others,others.1,postcode_district,year,quarter
0,49,38,50,NaN,AB1,2021,1
1,8150,269,688,NaN,AB10,2021,1
2,6476,204,1288,NaN,AB11,2021,1
3,15056,507,2781,NaN,AB12,2021,1
4,1885,48,132,NaN,AB13,2021,1


## NaN Values

In [14]:
nan = Preprocessing.NanAnalyzer(df)
nan.printNaNInfo()


NAN INFO IN THE DATASET:

No of columns with only NaN values: 1
Column list with only NaN values:
['others.1']

No of columns with partial NaN values: 3
Column list with partial NaN values:
['cars', 'motorcycles', 'others']

No of columns with no NaN values: 3
Column list with no NaN values:
['postcode_district', 'year', 'quarter']

Total Values in dataset: 650720

Total NaN Values in the dataset: 102449

Total Partial NaN Values in the dataset: 9489

Partial NaN Values Info:
       Columns  Partial_NaN_Count  NaN_Vals_By_Total_Vals
0         cars               3163                   0.034
1  motorcycles               3163                   0.034
2       others               3163                   0.034

Total NaN Values/Total Datset Values: 15.74 %


In [15]:
df.drop(columns=nan.getDict()['only_nan_cols'], inplace=True)

In [16]:
nan = Preprocessing.NanAnalyzer(df)
nan.printNaNInfo()


NAN INFO IN THE DATASET:

No of columns with only NaN values: 0
Column list with only NaN values:
[]

No of columns with partial NaN values: 3
Column list with partial NaN values:
['cars', 'motorcycles', 'others']

No of columns with no NaN values: 3
Column list with no NaN values:
['postcode_district', 'year', 'quarter']

Total Values in dataset: 557760

Total NaN Values in the dataset: 9489

Total Partial NaN Values in the dataset: 9489

Partial NaN Values Info:
       Columns  Partial_NaN_Count  NaN_Vals_By_Total_Vals
0         cars               3163                   0.034
1  motorcycles               3163                   0.034
2       others               3163                   0.034

Total NaN Values/Total Datset Values: 1.7 %


#### 2021 data doesn't have q4 data, hence those values are nan. For now the values are imputed with -1.

In [17]:
df.loc[(df['year'] == '2021') & (df['quarter'] == '4'), ['cars', 'motorcycles', 'others']] = -1
df.loc[(df['year'] == '2021') & (df['quarter'] == '4')]

,cars,motorcycles,others,postcode_district,year,quarter
69720,-1,-1,-1,AB1,2021,4
69721,-1,-1,-1,AB10,2021,4
69722,-1,-1,-1,AB11,2021,4
69723,-1,-1,-1,AB12,2021,4
69724,-1,-1,-1,AB13,2021,4
...,...,...,...,...,...,...
72618,-1,-1,-1,YO7,2021,4
72619,-1,-1,-1,YO8,2021,4
72620,-1,-1,-1,ZE1,2021,4
72621,-1,-1,-1,ZE2,2021,4


In [18]:
nan = Preprocessing.NanAnalyzer(df)
nan.printNaNInfo()


NAN INFO IN THE DATASET:

No of columns with only NaN values: 0
Column list with only NaN values:
[]

No of columns with partial NaN values: 3
Column list with partial NaN values:
['cars', 'motorcycles', 'others']

No of columns with no NaN values: 3
Column list with no NaN values:
['postcode_district', 'year', 'quarter']

Total Values in dataset: 557760

Total NaN Values in the dataset: 780

Total Partial NaN Values in the dataset: 780

Partial NaN Values Info:
       Columns  Partial_NaN_Count  NaN_Vals_By_Total_Vals
0         cars                260                   0.003
1  motorcycles                260                   0.003
2       others                260                   0.003

Total NaN Values/Total Datset Values: 0.14 %


In [19]:
df = df.drop(df[df['cars'].isnull() & df['motorcycles'].isnull() & df['others'].isnull()].index).reset_index(drop=True)

In [21]:
nan = Preprocessing.NanAnalyzer(df)
nan.printNaNInfo()
del nan


NAN INFO IN THE DATASET:

No of columns with only NaN values: 0
Column list with only NaN values:
[]

No of columns with partial NaN values: 0
Column list with partial NaN values:
[]

No of columns with no NaN values: 6
Column list with no NaN values:
['cars', 'motorcycles', 'others', 'postcode_district', 'year', 'quarter']

Total Values in dataset: 556200

Total NaN Values in the dataset: 0

Total Partial NaN Values in the dataset: 0

Partial NaN Values Info:
Empty DataFrame
Columns: [Columns, Partial_NaN_Count, NaN_Vals_By_Total_Vals]
Index: []

Total NaN Values/Total Datset Values: 0.0 %


In [20]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92700 entries, 0 to 92699
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   cars               92700 non-null  object
 1   motorcycles        92700 non-null  object
 2   others             92700 non-null  object
 3   postcode_district  92700 non-null  object
 4   year               92700 non-null  object
 5   quarter            92700 non-null  object
dtypes: object(6)
memory usage: 4.2+ MB


,cars,motorcycles,others,postcode_district,year,quarter
0,49,38,50,AB1,2021,1
1,8150,269,688,AB10,2021,1
2,6476,204,1288,AB11,2021,1
3,15056,507,2781,AB12,2021,1
4,1885,48,132,AB13,2021,1


## DataType Check

In [22]:
{c: '' for c in df.columns}

{'cars': '',
 'motorcycles': '',
 'others': '',
 'postcode_district': '',
 'year': '',
 'quarter': ''}

In [23]:
dtype_mapper = {'cars': 'int64',
                'motorcycles': 'int64',
                'others': 'int64',
                'year': 'int64',
                'postcode_district': 'str',
                'quarter': 'int64'}

In [24]:
dta = Preprocessing.DataTypeAnalyzer()

dta.findUnExpectedDtypeValues(df, dtype_mapper)

,Columns,ExpectedDtype,MismatchedValues
0,cars,int64,[c]
1,motorcycles,int64,[c]
2,others,int64,[c]
3,year,int64,[]
4,postcode_district,str,[]
5,quarter,int64,[]


#### To convert dataframe with desired datatypes we only need to handle 'c' values which signifies that the number of vehicles registered at that postcode or district is very low. So imputing c with 1 to signify low count.

In [25]:
for col in ['cars', 'motorcycles', 'others']:
    df[col].replace({'c':1}, inplace=True)

In [26]:
dta.findUnExpectedDtypeValues(df, dtype_mapper)

,Columns,ExpectedDtype,MismatchedValues
0,cars,int64,[]
1,motorcycles,int64,[]
2,others,int64,[]
3,year,int64,[]
4,postcode_district,str,[]
5,quarter,int64,[]


In [33]:
df = dta.convertDtypes(df, {k:v if v != 'str' else 'object' for k, v in dtype_mapper.items()}).reset_index(drop=True)

In [34]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92700 entries, 0 to 92699
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   cars               92700 non-null  int64 
 1   motorcycles        92700 non-null  int64 
 2   others             92700 non-null  int64 
 3   postcode_district  92700 non-null  object
 4   year               92700 non-null  int64 
 5   quarter            92700 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 4.2+ MB


,cars,motorcycles,others,postcode_district,year,quarter
0,49,38,50,AB1,2021,1
1,8150,269,688,AB10,2021,1
2,6476,204,1288,AB11,2021,1
3,15056,507,2781,AB12,2021,1
4,1885,48,132,AB13,2021,1


## Duplicates

In [37]:
Preprocessing.DuplicateAnalyzer(df, df.columns.tolist()).printDuplicateInfo()


Matching columns count: 0

Matching columns dict:
{}

Duplicate Row count: 0

DUplicate Rows:


,cars,motorcycles,others,postcode_district,year,quarter


## Finalizing and Saving

In [41]:
df.head(10)

,cars,motorcycles,others,postcode_district,year,quarter
0,49,38,50,AB1,2021,1
1,8150,269,688,AB10,2021,1
2,6476,204,1288,AB11,2021,1
3,15056,507,2781,AB12,2021,1
4,1885,48,132,AB13,2021,1
5,2604,95,290,AB14,2021,1
6,20201,483,1172,AB15,2021,1
7,10470,337,1419,AB16,2021,1
8,37,33,40,AB2,2021,1
9,13462,491,2698,AB21,2021,1


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92700 entries, 0 to 92699
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   cars               92700 non-null  int64 
 1   motorcycles        92700 non-null  int64 
 2   others             92700 non-null  int64 
 3   postcode_district  92700 non-null  object
 4   year               92700 non-null  int64 
 5   quarter            92700 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 4.2+ MB


In [43]:
df.agg(['count', 'min', 'max'])

,cars,motorcycles,others,postcode_district,year,quarter
count,92700,92700,92700,92700,92700,92700
min,-1,-1,-1,AB1,2014,1
max,283799,2886,90001,ZE3,2021,4


In [48]:
"/".join(project_dir.split("/")[:-1]) + "/licensed_vehicles_by_postcode_district_and_body_type.csv"

'/Users/gurdeep/Documents/tb2/DSMP/licensed_vehicles_by_postcode_district_and_body_type.csv'

In [49]:
df.to_csv("/".join(project_dir.split("/")[:-1]) + "/licensed_vehicles_by_postcode_district_and_body_type.csv", index=False)